In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV,KFold,train_test_split

/home/suraj/anaconda3/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [2]:
data = pd.read_csv('diabetes.csv',sep=',')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
sum(data['Glucose']==0)

5

In [7]:
data.shape

(768, 9)

In [8]:
data.columns

Index([u'Pregnancies', u'Glucose', u'BloodPressure', u'SkinThickness',
       u'Insulin', u'BMI', u'DiabetesPedigreeFunction', u'Age', u'Outcome'],
      dtype='object')

In [3]:
columns = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for cols in columns:
    data[cols].replace(0,np.NaN,inplace=True)
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
data.dropna(inplace=True)

In [12]:
data.shape

(392, 9)

In [5]:
X = data.drop('Outcome',axis=1)
y = data['Outcome']

In [23]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53
13,1,189.0,60.0,23.0,846.0,30.1,0.398,59


In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaled= StandardScaler()

In [8]:
X_standardised = scaled.fit_transform(X)

In [9]:
X_scaled = pd.DataFrame(X_standardised,columns=X.columns)

In [10]:
X_scaled.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [16]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV,KFold,train_test_split
def create_model(learn_rate):
    model = Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #Compile the model
    adam = Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
    return model
#model=create_model()
#print(model.summary())

estimator = KerasClassifier(build_fn=create_model,verbose=1)
param_grid = {'epochs':[20,50,100],'batch_size':[10,20,40],'learn_rate':[0.1,0.01,0.001]}
grid = GridSearchCV(estimator,param_grid=param_grid,cv=KFold(random_state=6),verbose=10)
grid_results = grid.fit(X_scaled,y)
print("Best:{0}, using {1}".format(grid_results.best_score_,grid_results.best_params_))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] epochs=20, learn_rate=0.1, batch_size=10 ........................


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Epoch 1/20
261/261 [==============================] - 0s 2ms/step - loss: 0.4889 - acc: 0.7088
Epoch 2/20
261/261 [==============================] - 0s 394us/step - loss: 0.5058 - acc: 0.7586
Epoch 3/20
261/261 [==============================] - 0s 381us/step - loss: 0.4561 - acc: 0.7969
Epoch 4/20
261/261 [==============================] - 0s 387us/step - loss: 0.4098 - acc: 0.7969
Epoch 5/20
261/261 [==============================] - 0s 493us/step - loss: 0.4053 - acc: 0.8084
Epoch 6/20
261/261 [==============================] - 0s 381us/step - loss: 0.3744 - acc: 0.8123
Epoch 7/20
261/261 [==============================] - 0s 330us/step - loss: 0.4120 - acc: 0.8123
Epoch 8/20
261/261 [==============================] - 0s 407us/step - loss: 0.3790 - acc: 0.8238
Epoch 9/20
261/261 [==============================] - 0s 305us/step - loss: 0.3698 - acc: 0.8238
Epoch 10/20
261/261 [==============================] - 0s 295us/step - loss: 0.3927 - acc: 0.8314 0s - loss: 0.3679 - acc: 0.833


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   34.3s remaining:    0.0s


Epoch 1/20
261/261 [==============================] - 0s 132us/step - loss: 0.5342 - acc: 0.7011
Epoch 2/20
261/261 [==============================] - 0s 225us/step - loss: 0.4933 - acc: 0.7586
Epoch 3/20
261/261 [==============================] - 0s 181us/step - loss: 0.5125 - acc: 0.7471
Epoch 4/20
261/261 [==============================] - 0s 134us/step - loss: 0.4797 - acc: 0.7510
Epoch 5/20
261/261 [==============================] - 0s 142us/step - loss: 0.4965 - acc: 0.7548
Epoch 6/20
261/261 [==============================] - 0s 173us/step - loss: 0.4480 - acc: 0.8008
Epoch 7/20
261/261 [==============================] - 0s 169us/step - loss: 0.4530 - acc: 0.8199
Epoch 8/20
261/261 [==============================] - 0s 137us/step - loss: 0.4576 - acc: 0.7969
Epoch 9/20
261/261 [==============================] - 0s 158us/step - loss: 0.4097 - acc: 0.8084
Epoch 10/20
261/261 [==============================] - 0s 182us/step - loss: 0.4297 - acc: 0.8084
Epoch 11/20
261/261 [========

...

261/261 [==============================] - 0s 79us/step - loss: 0.3658 - acc: 0.8314
Epoch 96/100
261/261 [==============================] - 0s 65us/step - loss: 0.3654 - acc: 0.8314
Epoch 97/100
261/261 [==============================] - 0s 60us/step - loss: 0.3652 - acc: 0.8276
Epoch 98/100
261/261 [==============================] - 0s 61us/step - loss: 0.3649 - acc: 0.8276
Epoch 99/100
261/261 [==============================] - 0s 59us/step - loss: 0.3646 - acc: 0.8276
Epoch 100/100
261/261 [==============================] - 0s 24us/step
[CV]  epochs=100, learn_rate=0.001, batch_size=40, score=0.709923655932, total=   6.6s
[CV] epochs=100, learn_rate=0.001, batch_size=40 .....................
Epoch 1/100
261/261 [==============================] - 0s 95us/step - loss: 0.6915 - acc: 0.6552
Epoch 2/100
261/261 [==============================] - 0s 90us/step - loss: 0.6877 - acc: 0.6590
Epoch 3/100
261/261 [==============================] - 0s 127us/step - loss: 0.6838 - acc: 0.6552
Epo

261/261 [==============================] - 0s 86us/step - loss: 0.4215 - acc: 0.8046
Epoch 75/100
261/261 [==============================] - 0s 77us/step - loss: 0.4210 - acc: 0.8046
Epoch 76/100
261/261 [==============================] - 0s 65us/step - loss: 0.4206 - acc: 0.8046
Epoch 77/100
261/261 [==============================] - 0s 86us/step - loss: 0.4202 - acc: 0.8046
Epoch 78/100
261/261 [==============================] - 0s 63us/step - loss: 0.4201 - acc: 0.8046
Epoch 79/100
261/261 [==============================] - 0s 85us/step - loss: 0.4194 - acc: 0.8046
Epoch 80/100
261/261 [==============================] - 0s 79us/step - loss: 0.4190 - acc: 0.8046
Epoch 81/100
261/261 [==============================] - 0s 75us/step - loss: 0.4186 - acc: 0.8046
Epoch 82/100
261/261 [==============================] - 0s 67us/step - loss: 0.4184 - acc: 0.8046
Epoch 83/100
261/261 [==============================] - 0s 61us/step - loss: 0.4179 - acc: 0.8046
Epoch 84/100
261/261 [===========

262/262 [==============================] - 0s 66us/step - loss: 0.5857 - acc: 0.7824
Epoch 55/100
262/262 [==============================] - 0s 48us/step - loss: 0.5842 - acc: 0.7824
Epoch 56/100
262/262 [==============================] - 0s 64us/step - loss: 0.5825 - acc: 0.7901
Epoch 57/100
262/262 [==============================] - 0s 59us/step - loss: 0.5810 - acc: 0.7939
Epoch 58/100
262/262 [==============================] - 0s 76us/step - loss: 0.5795 - acc: 0.7977
Epoch 59/100
262/262 [==============================] - 0s 63us/step - loss: 0.5779 - acc: 0.7977
Epoch 60/100
262/262 [==============================] - 0s 61us/step - loss: 0.5763 - acc: 0.7939
Epoch 61/100
262/262 [==============================] - 0s 65us/step - loss: 0.5746 - acc: 0.7901
Epoch 62/100
262/262 [==============================] - 0s 99us/step - loss: 0.5734 - acc: 0.7939
Epoch 63/100
262/262 [==============================] - 0s 57us/step - loss: 0.5718 - acc: 0.7939
Epoch 64/100
262/262 [===========

[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  8.6min finished


Epoch 1/50
392/392 [==============================] - 0s 73us/step - loss: 0.6153 - acc: 0.6709
Epoch 2/50
392/392 [==============================] - 0s 68us/step - loss: 0.4951 - acc: 0.6684
Epoch 3/50
392/392 [==============================] - 0s 68us/step - loss: 0.4833 - acc: 0.6684
Epoch 4/50
392/392 [==============================] - 0s 86us/step - loss: 0.4691 - acc: 0.7832
Epoch 5/50
392/392 [==============================] - 0s 80us/step - loss: 0.4597 - acc: 0.7908
Epoch 6/50
392/392 [==============================] - 0s 80us/step - loss: 0.4538 - acc: 0.7985
Epoch 7/50
392/392 [==============================] - 0s 90us/step - loss: 0.4463 - acc: 0.7934
Epoch 8/50
392/392 [==============================] - 0s 89us/step - loss: 0.4377 - acc: 0.7985
Epoch 9/50
392/392 [==============================] - 0s 97us/step - loss: 0.4323 - acc: 0.8010
Epoch 10/50
392/392 [==============================] - 0s 99us/step - loss: 0.4286 - acc: 0.7908
Epoch 11/50
392/392 [==================

In [17]:
y_pred = grid.predict(X_scaled)

392/392 [==============================] - 0s 29us/step


In [18]:
print(classification_report(y,y_pred))

             precision    recall  f1-score   support

          0       0.94      0.84      0.89       262
          1       0.74      0.89      0.81       130

avg / total       0.87      0.86      0.86       392

